# Loading a semantic map using knowrob

This notebook demonstrates how to load a semantic map into KnowRob

### Importing KnowRob Modules

We will first initialize KnowRob

In [ ]:
import json
from knowrob import *
    
InitKnowRob()

### Setting Up Knowledge Base

Now we set up the knowledge base. See the second data-sources field, to see the path to an expected ontology. Add this ontology before running this cell.

In [ ]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
            {"alias": "dul", "uri": "http://www.ontologydesignpatterns.org/ont/dul/DUL.owl"},
			{"alias": "USD", "uri": "https://ease-crc.org/ont/USD.owl"},
            {"alias": "dfl", "uri": "http://www.ease-crc.org/ont/SOMA_DFL.owl"}
		]
	},
	"data-sources": [
		{"path": "owl/USD.owl", "format": "rdf-xml"},
		{"path": "owl/house_4.owl", "format": "rdf-xml"}
	],
	"data-backends": [
    {
      "type": "Redland",
      "name": "redland",
      "read-only": False
    }
	],  
    "reasoner": [
    {
      "type": "SOMADFLReasoner",
      "name": "DFLReasoner",
      "module": "reasoner/dfl_reasoner.py",
      "data-backend": "redland"
    }
    ]
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

def runQuery(queryStr):
    phi = QueryParser.parse(queryStr)
    resultStream = kb.submitQuery(phi, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
    resultQueue = resultStream.createQueue()
    retq = None
    result = resultQueue.pop_front()
    while not result.indicatesEndOfEvaluation():
        if retq is None:
            retq = []
        if isinstance(result, AnswerYes):
            aux = {}
            for substitution in result.substitution():
                variable, term = substitution[1], substitution[2]
                aux[str(variable)] = str(term)
            retq.append(aux)
        elif isinstance(result, AnswerNo):
            retq = None
            break
        result = resultQueue.pop_front()
    return retq


### Submitting the Competency Questions

Now we we will run the competency questions.

#### CQ1: Which objects do I need for breakfast?

In [ ]:
# CQ1: which objects do I need for breakfast? -- reinterpreted as, what objects are subclasses of breakfast food, 
# and what can I use to serve those

# Are there any objects that are instances of breakfast food?
bdgs = runQuery("dfl:isInstanceOf(?x, 'http://www.ease-crc.org/ont/SOMA_DFL.owl#breakfast_food.n.wn.food')")
breakfastFoodItems = [x["?x"] for x in (bdgs or [])]

# Are there any objects that contain something that is a breakfast food?

#     First, what kinds of breakfast food are there?
bdgs = runQuery("dfl:isSubclassOf(?x, 'http://www.ease-crc.org/ont/SOMA_DFL.owl#breakfast_food.n.wn.food')")
breakfastFoodClasses = [x["?x"] for x in (bdgs or [])]

#     Second, what containers for these things are around?
containerItems = set()
breakfastFoodClassesPresent = set()
for food in breakfastFoodClasses:
    bdgs = runQuery("dfl:hasPart(?x, '%s')" % food)
    containerItems = containerItems.union([x["?x"] for x in (bdgs or [])])
    if (bdgs is not None) and (0 < len(bdgs)):
        breakfastFoodClassesPresent.add(food)

# What sort of tools are used to serve the breakfast food we found 
#     (either as stand-alone items, or contained somewhere)?
toolItems = set()
for food in set().union(breakfastFoodItems).union(breakfastFoodClassesPresent):
    bdgs = runQuery("dfl:useMatch('http://www.ease-crc.org/ont/SOMA_DFL.owl#serve.v.wn.consumption..concrete', ?x, '%s')" % food)
    toolItems = toolItems.union([x["?x"] for x in (bdgs or [])])

# Report the results
edibles = containerItems.union(breakfastFoodItems)
print("These things look like breakfast food or contain some breakfast food: %s" % str(sorted(edibles)))
print(" ... and might need these items to serve them: %s" % str(sorted(list(toolItems))))


#### CQ2 Which objects contain something to drink?

In [ ]:
# CQ2: Which objects contain something to drink?

# What kinds of beverages are there?
bdgs = runQuery("dfl:isSubclassOf(?x, 'http://www.ease-crc.org/ont/SOMA_DFL.owl#beverage.n.wn.food')")
beverageClasses = [x["?x"] for x in (bdgs or [])]

# What containers for these things are around?
containerItems = set()
for drink in beverageClasses:
    bdgs = runQuery("dfl:hasPart(?x, '%s')" % drink)
    containerItems = containerItems.union([x["?x"] for x in (bdgs or [])])

#### Assuming conjunctive queries work
# bdgs = runQuery("dfl:isSubclassOf(?c, 'http://www.ease-crc.org/ont/SOMA_DFL.owl#beverage.n.wn.food'), \
#                  dfl:hasPart(?x, ?c)")
# containerItems = set([x["?x"] for x in bdgs])

# Report results
print("These items should contain something to drink: %s" % str(sorted(list(containerItems))))


#### CQ3 Where do we expect an item to be?

In [ ]:
# CQ3: Where do we expect an item to be? -- note, there is no ranking of locations by likelihood,
# and there may be many plausible locations!

# HAXX
toolItems = ["https://ease-crc.org/ont/USD.owl#Fork_6_30"]

# For example, lets pick a tool from CQ1
if 0 < len(toolItems):
    item = list(toolItems)[0]

    bdgs = runQuery("dfl:useMatch('http://www.ease-crc.org/ont/SOMA_DFL.owl#store.v.wn.possession..place', ?x, '%s')" % item)
    locations = [x["?x"] for x in (bdgs or [])]

    # Report results
    print("The %s might be in one of these places: %s" % (item, str(sorted(list(locations)))))
else:
    print("Did not find any tools at CQ1 so will skip CQ3 ...")



#### CQ4 What can I grasp on an object to open it?

In [ ]:
# CQ4: What can I grasp on an object to open it?

# For example, lets pick a tool from CQ1
if 0 < len(toolItems):
    item = list(toolItems)[0]

    # What parts does the object have?
    bdgs = runQuery("dul:hasPart('%s', ?x)" % item)
    parts = [x["?x"] for x in (bdgs or [])]
    parts = set(parts).union([item])
    
    # Which of them are graspable?
    graspables = set()
    for part in parts:
        bdgs = runQuery("dfl:hasDisposition('%s', 'dfl:hold.v.wn.contact..grasp.Theme')" % (part))
        if bdgs is not None:
            graspables.add(part)
    # IF complex queries work (and assuming an object has itself as part, or else a disjunction is needed):
    # bdgs = runQuery(f"dul:hasPart('{item}', ?x), dfl:hasDisposition(?x, 'dfl:hold.v.wn.contact..grasp')")
    # graspables = [x["?x"] for x in bdgs]
    # Report results
    print("Can try to grasp %s by grasping one of %s" % (item, str(sorted(list(graspables)))))
else:
    print("Did not find any tools at CQ1 so will skip CQ4 ...")

#### CQ5 Where should I put the utensils for breakfast?

In [ ]:
# CQ5: Where to place the utensils for breakfast?

bdgs = runQuery("dfl:hasDisposition(?x, 'http://www.ease-crc.org/ont/SOMA_DFL.owl#serve.v.wn.consumption..concrete.Location')")
locations = [x["?x"] for x in (bdgs or [])]

# Report results
print("You could set up the meal at one of these locations: %s" % str(locations))